In [ ]:
def compute_terrain_corrected_irradiance(
    aod550, tcwv, elevation,
    solar_zenith_deg, slope_deg, aspect_deg, solar_azimuth_deg,
    svf=1.0, direct_ratio=0.8
):
    """
    Compute terrain-corrected clear-sky solar irradiance (W/m^2)
    using simplified Solis model + topographic corrections.

    Parameters:
    - aod550: Aerosol Optical Thickness at 550 nm
    - tcwv: Total column water vapor in kg/m^2
    - elevation: Elevation above sea level in meters
    - air_temperature: Air temperature in Kelvin (optional, not used in this version)
    - solar_zenith_deg: Solar zenith angle in degrees
    - slope_deg: Terrain slope in degrees
    - aspect_deg: Terrain aspect in degrees (0 = N, 180 = S)
    - solar_azimuth_deg: Solar azimuth angle in degrees
    - svf: Sky view factor (0 to 1), default = 1 (full sky visible)
    - direct_ratio: Fraction of direct radiation in total, default = 0.8

    Returns:
    - GHI (W/m^2): terrain-corrected global horizontal irradiance
    """
    G_sc = 1367.0  # Solar constant (W/m^2)
    deg2rad = np.pi / 180

    # Convert AOD to 700 nm using Angström exponent
    aod700 = aod550 * (700 / 550) ** -1.3
    tcwv_cm = tcwv / 10.0

    theta_z_rad = solar_zenith_deg * deg2rad
    slope_rad = slope_deg * deg2rad
    aspect_rad = aspect_deg * deg2rad
    solar_azimuth_rad = solar_azimuth_deg * deg2rad

    # Simplified transmittance model
    tau_clear = np.exp(-0.9 * aod700) * np.exp(-0.15 * tcwv_cm)
    ghi_flat = G_sc * np.cos(theta_z_rad) * tau_clear

    # Elevation correction
    ghi_elev = ghi_flat * (1 + 0.06 * (elevation / 1000.0))

    # Terrain incidence angle
    cos_incidence = (
        np.cos(theta_z_rad) * np.cos(slope_rad) +
        np.sin(theta_z_rad) * np.sin(slope_rad) * np.cos(solar_azimuth_rad - aspect_rad)
    )
    cos_incidence = np.maximum(cos_incidence, 0)

    # Direct and diffuse components
    direct_flat = ghi_elev * direct_ratio
    diffuse_flat = ghi_elev * (1 - direct_ratio)

    direct_terrain = direct_flat * cos_incidence
    diffuse_terrain = diffuse_flat * svf

    return direct_terrain + diffuse_terrain